In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5


In [4]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [5]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:5]

,tid,count,rating
166,166,3666,573.25
457,457,2042,162.00
6338,6338,1896,153.75
80,80,1835,412.25
1364,1364,1792,327.50


In [6]:
# Linear
size = int(len(pop_count) // 20)
pop_normalized = pd.DataFrame()
for i in range(20):
    if i == 0:
        pop = pop_count[:size].copy()
    elif i == 19:
        pop = pop_count[size*19:].copy()
    else:
        pop = pop_count[size*i:size*(i+1)].copy()
    pop['rating'] = (20 - i)*0.25
    pop_normalized = pop_normalized.append(pop)
pop_normalized = pop_normalized[['tid', 'rating']]
pop_normalized

,tid,rating
166,166,5.00
457,457,5.00
6338,6338,5.00
80,80,5.00
1364,1364,5.00
...,...,...
37553,37554,0.25
42281,42283,0.25
11800,11800,0.25
45731,45733,0.25


In [7]:
# # min_max
# max_rating = pop_count.iloc[0]['count']
# pop_normalized = pop_count.copy()
# pop_normalized = pop_normalized[['tid', 'count']]
# pop_normalized['rating'] = pop_normalized['count']
# pop_normalized['rating'] /= max_rating
# pop_normalized['rating'] *= 5
# pop_normalized[:5]

In [8]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [9]:
top_4_results = []
top_20_results = []
top_100_results = []

In [10]:
for lambda_ in tqdm(lambdas):
    top_n_size = 100
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    for top_n_size in [100, 20, 4]:
        avg_precisions = []
        for i in range(len(groups_n)):
            groups = groups_n[i]
            top_n_lists = top_n_lists_g[i]

            precisions = []
            for j in (range(len(groups))):

                group = groups[j]
                top_n_list = top_n_lists[j][:top_n_size]

                high_rating = 0

                for k in range(top_n_size):
                    for l in range(len(group)):
                        uid = group[l]
                        tid = top_n_list[k]
                        t = test[test['uid'] == uid]
                        t = t[t['tid'] == tid]
                        if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                            high_rating += 1
                            break

                precision = high_rating / top_n_size
                precisions.append(precision)

            avg_precision = 0
            for precision in precisions:
                avg_precision += precision
            avg_precision /= len(precisions)
            avg_precisions.append(avg_precision)
#         print(avg_precisions)
        if top_n_size == 4:
            top_4_results.append(avg_precisions)
        elif top_n_size == 20:
            top_20_results.append(avg_precisions)
        elif top_n_size == 100:
            top_100_results.append(avg_precisions)
    #     print(sum(avg_precisions) / len(avg_precisions))
    #     print('----------------------------------------') 

In [11]:
for i in top_4_results:
    print(i)
print('------------------')

for i in top_20_results:
    print(i)
print('------------------')

for i in top_100_results:
    print(i)

[0.09333333333333334, 0.11363636363636363, 0.09210526315789473, 0.09615384615384616]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.2866666666666667, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.29, 0.24242424242424243, 0.34210526315789475, 0.28846153846153844]
[0.29, 0.23484848484848486, 0.34210526315789475, 0.28846153846153844]
[0.27666666666666667, 0.24242424242424243, 0.3157894736842105, 0.3076923076923077]
------------------
[0.0533333333333333, 0.07424242424242422, 0.06842105263157898, 0.06538461538461539]
[